In [1]:
from datasets import load_dataset
import re
import random
import json
import random
import numpy as np

In [ ]:
personas = []
with open('/Users/venkat/Downloads/persona.jsonl', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        personas.append(json.loads(line)['persona'])

In [ ]:
personas[0]

In [ ]:
random.seed(2)
sample = random.sample(personas, 100)

In [ ]:
with open('data/sample_personas.txt', 'w') as f:
    for persona in sample:
        f.write(persona+"\n")

In [2]:
with open('data/sample_personas.txt', 'r') as f:
    personas = [x.strip() for x in f.readlines()]

## Generating subjective questions from Homogeniety Bias in LLMs paper

In [ ]:
num_iter = 1
race_list = np.array(['African', 'Asian', 'Hispanic', 'White'])
gender_list = np.array(['man', 'woman'])
format_list = np.array(['story about', 'character description of', 'biography of', 
  'introduction of', 'social media profile of', 'synopsis for', 'narrative of', 
  'self-introduction of', 'tragic story about', 'funny story about', 'romantic story about', 
  'horror story about', 'dramatic story about'])

race_column = np.repeat(race_list, num_iter * len(gender_list) * len(format_list), axis = 0)
gender_column = np.tile(np.repeat(gender_list, num_iter * len(format_list)), len(race_list))
format_column = np.tile(np.repeat(format_list, num_iter), len(race_list) * len(gender_list))

In [ ]:
with open('data/subj_questions.txt', 'w') as f:
    for format_index in range(len(format_list)):
        f.write("Write a thirty-word {format} a(n)".
           format(format = format_list[format_index], race = race_list[race_index], gender = gender_list[gender_index])+"\n")

## Dolly prompts

In [3]:
dataset = load_dataset("databricks/databricks-dolly-15k")["train"]
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

In [5]:
new_df = dataset.filter(lambda row: row['category']=='creative_writing').to_pandas().sample(n=100, random_state=2)
new_df['num_tokens'] = new_df.response.apply(lambda x: len(tokenizer.encode(x)))

### Also find number of words approximately and round up to nearest 10

In [6]:
from nltk.tokenize import word_tokenize

In [7]:
len([x for x in word_tokenize(new_df.response.values[0], preserve_line=True) if x.isalnum()])

177

In [9]:
new_df['num_words_round'] = new_df.response.apply(lambda x: round(len([w for w in word_tokenize(x, preserve_line=True) if w.isalnum()]), -1))

In [11]:
new_df['num_tokens_round'] = new_df.num_tokens.apply(lambda x: round(x,-1))

In [12]:
new_df.head()

,instruction,context,response,category,num_tokens,num_tokens_round,num_words_round
525,Please propose an argument to convince my moth...,,"Dear Mum, I would like to talk to you about th...",creative_writing,211,210,180
172,Write a paragraph to refute a claim by a colle...,,There are several strong arguments against con...,creative_writing,189,190,160
109,What is the best way to answer an interview qu...,,The first recommended step is to ask clarifyin...,creative_writing,128,130,100
37,Write the first paragraph of an advertising b...,,"This fantastic hotel is ideally located, minut...",creative_writing,89,90,60
295,"Write an intro to a meetup about music, medici...",,"Music, Medicine, and Machines\n\nJoin us for a...",creative_writing,313,310,220


In [14]:
new_df[new_df.num_words_round>250].shape

(22, 7)

In [15]:
new_df.loc[:, ['instruction', 'num_tokens', 'num_tokens_round', 'num_words_round']].to_csv('data/dolly_creative_prompts_sample.tsv', sep='\t', index=True)

In [ ]:
new_df = dataset.filter(lambda row: row['category']=='brainstorming').to_pandas().sample(n=100, random_state=2)
new_df